## DW state

In [ ]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d # Hilbert space spin basis
import numpy as np # generic math functions
import matplotlib.pyplot as plt # plotting library

In [ ]:
### floquet modulation
def period(n):
    fib_list = ["a", "b"]*n
    return fib_list

L=18
Jr=1.0 # spin=spin interaction
TT=0.1  ##实际的floquet调制
step=2
order=100

s=period(order)
basis = spin_basis_1d(L,pauli=False,m=0) 
i0 = basis.index("010101010101010101") 
psi01 = np.zeros(basis.Ns,dtype=np.float64)
psi01[i0] = 1.0 
psi0=(psi01)###定义初态



no_checks = dict(check_herm=False,check_symm=False,check_pcon=False)
n_list = [hamiltonian([["z",[[1.0,i]]]],[],basis=basis,dtype=np.float64,**no_checks) for i in range(L)]
Jr=1# spin ring
H0=[[-Jr,  4*i,(4*i+1),(4*i+2),(4*i+3)]  for i in range(L//4)] 
H1=[[-Jr,  4*i+2,(4*i+3),(4*i+4)%L,(4*i+5)%L]  for i in range(L//4-1)] 
 
#print(H0,H1)

# static and dynamic lists
static0=[["+--+",H0],["-++-",H0]]
static1=[["+--+",H1],["-++-",H1]]
dynamic=[]
###### construct Hamiltonian
Hd0=hamiltonian(static0,dynamic,basis=basis,dtype=np.float64,**no_checks)
Hd1=hamiltonian(static1,dynamic,basis=basis,dtype=np.float64,**no_checks)

psitot=psi0
psiini=psi0
ent=[]
fide=[]
sd=[]
expt_n_t_sta1=[]
for i in range(len(s)):
    
    if i>0:
        psiini=psi_list[:,-1]
    if s[i] == 'a':
        tmax0=TT
        tlist0=np.linspace(0,tmax0,step)
        s[i] = tlist0
        psi_list=Hd0.evolve(psiini,0,tlist0)
        #ent=[]
        #fide=[]
    elif s[i] == 'b':
        tmax1=TT
        tlist1=np.linspace(0,tmax1,step)
        s[i] = tlist1
        psi_list=Hd1.evolve(psiini,0,tlist1)
        #ent=[]
        #fide=[]
        
    expt_n_t_sta1.append([n.expt_value(psi_list[:,-1]).real for n in n_list])

    print('i = % 3d | % 3d,'%(i, len(s)),end='\r')
            
tlisttot=[0]#总时间序列
tlist=[0]
for i in range(len(s)):
    tlist=tlist[-1]+s[i]
    tlisttot=np.r_[tlisttot,tlist]
    
import pandas as pd


df = pd.DataFrame(expt_n_t_sta1)

df.to_csv('dw4f.csv', index=False)

In [ ]:
###effective model
L=18 ###等效模型
basis = spin_basis_1d(L,pauli=False,m=0) 
i0 = basis.index("010101010101010101") # pick state from basis set
psi01 = np.zeros(basis.Ns,dtype=np.float64)
psi01[i0] = 1.0 # define state corresponding to the string "111000"
psi0=(psi01)

Jr=1# spin ring
H0=[[-Jr,  4*i,(4*i+1),(4*i+2),(4*i+3)]  for i in range(L//4)] 
H1=[[-Jr,  4*i+2,(4*i+3),(4*i+4)%L,(4*i+5)%L]  for i in range(L//4)] 
print(H0,H1)
no_checks = dict(check_herm=False,check_symm=False,check_pcon=False)
static0=[["+--+",H0],["-++-",H0],["+--+",H1],["-++-",H1]]

dynamic=[]
###### construct Hamiltonian
Hd0=hamiltonian(static0,dynamic,basis=basis,**no_checks)
#
ttol=10
step=100
tlist1=np.linspace(0,ttol,step)

psi_list=Hd0.evolve(psi0,0,tlist1)

n_list = [hamiltonian([["z",[[1.0,i]]]],[],basis=basis,dtype=np.float64,**no_checks) for i in range(L)]
expt_n_t_sta1 = np.vstack([n.expt_value(psi_list).real for n in n_list]).T
#print(expt_n_t_sta,psi_k)
for i in range(L//2):
    plt.plot(tlist1,expt_n_t_sta1[:,2*i],label=i)
plt.legend(loc=[1.01,0])
plt.ylim([-0.52,0.52])
aaa=[]
for i in range(L):
    aaa.append(expt_n_t_sta1[:,i])
import pandas as pd


df = pd.DataFrame(aaa)

df.to_csv('dw4e.csv', index=False)

In [ ]:
##XX model
L=18# XX模型
basis = spin_basis_1d(L,pauli=False,m=0) 
#i0 = basis.index("0110010101") # pick state from basis set


i0 = basis.index("010101010101010101") # pick state from basis set
psi01 = np.zeros(basis.Ns,dtype=np.float64)
psi01[i0] = 1.0 # define state corresponding to the string "111000"
psi0=(psi01)

Jp=1# spin ring
h=0
##### construct basis in the 0-total momentum and +1-parity sector
# define PBC site-coupling lists for operators
H0=[[-Jp,i,(i+2)%L]  for i in range(L)] 
H1=[[-Jp,2*i,(2*i+1)]  for i in range(L//2)] 


H01=[[h*i,2*i] for i in range(L//2)]
H02=[[h*(i+1),2*i+1] for i in range(L//2)]

print(H01,H02)
no_checks = dict(check_herm=False,check_symm=False,check_pcon=False)
# static and dynamic lists
static0=[["+-",H0],["-+",H0],["+-",H1],["-+",H1], ["z",H01],["z",H02]]

dynamic=[]
###### construct Hamiltonian
Hd0=hamiltonian(static0,dynamic,basis=basis,**no_checks)
#
ttol=10
step=100
tlist1=np.linspace(0,ttol,step)

psi_list=Hd0.evolve(psi0,0,tlist1)

n_list = [hamiltonian([["z",[[1.0,i]]]],[],basis=basis,dtype=np.float64,**no_checks) for i in range(L)]
expt_n_t_sta1 = np.vstack([n.expt_value(psi_list).real for n in n_list]).T
#print(expt_n_t_sta,psi_k)
for i in range(L//2):
    plt.plot(tlist1,expt_n_t_sta1[:,2*i],label=i)
plt.legend(loc=[1.01,0])
plt.ylim([-0.52,0.52])
aaa=[]
for i in range(L):
    aaa.append(expt_n_t_sta1[:,i])
import pandas as pd

df = pd.DataFrame(aaa)
df.to_csv('dw2.csv', index=False)

## NEEL state

In [ ]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d # Hilbert space spin basis
import numpy as np # generic math functions
import matplotlib.pyplot as plt # plotting library

In [ ]:
def period(n):
    fib_list = ["a", "b"]*n
    return fib_list

L=18
Jr=1.0 # spin=spin interaction

In [ ]:
### floquet modulation
TT=0.1
step=2
order=100

s=period(order)
basis = spin_basis_1d(L,pauli=False,m=0) 
i0 = basis.index("011001100110011001") # pick state from basis set
psi01 = np.zeros(basis.Ns,dtype=np.float64)
psi01[i0] = 1.0 # define state corresponding to the string "111000"
psi0=(psi01)
no_checks = dict(check_herm=False,check_symm=False,check_pcon=False)
n_list = [hamiltonian([["z",[[1.0,i]]]],[],basis=basis,dtype=np.float64,**no_checks) for i in range(L)]


Jr=1# spin ring

H0=[[-Jr,  4*i,(4*i+1),(4*i+2),(4*i+3)]  for i in range(L//4)] 
H1=[[-Jr,  4*i+2,(4*i+3),(4*i+4)%L,(4*i+5)%L]  for i in range(L//4)] 
 
#print(H0,H1)

# static and dynamic lists
static0=[["+--+",H0],["-++-",H0]]
static1=[["+--+",H1],["-++-",H1]]
dynamic=[]
###### construct Hamiltonian
Hd0=hamiltonian(static0,dynamic,basis=basis,dtype=np.float64,**no_checks)
Hd1=hamiltonian(static1,dynamic,basis=basis,dtype=np.float64,**no_checks)

psitot=psi0
psiini=psi0
ent=[]
fide=[]
sd=[]
expt_n_t_sta1=[]
for i in range(len(s)):
    
    if i>0:
        psiini=psi_list[:,-1]
    if s[i] == 'a':
        tmax0=TT
        tlist0=np.linspace(0,tmax0,step)
        s[i] = tlist0[-1]
        psi_list=Hd0.evolve(psiini,0,tlist0)
        #ent=[]
        #fide=[]
    elif s[i] == 'b':
        tmax1=TT
        tlist1=np.linspace(0,tmax1,step)
        s[i] = tlist1[-1]
        psi_list=Hd1.evolve(psiini,0,tlist1)
        #ent=[]
        #fide=[]
        
    expt_n_t_sta1.append(np.array([n.expt_value(psi_list[:,-1]).real for n in n_list]))

    print('i = % 3d | % 3d,'%(i, len(s)),end='\r')
            

expt_n_t_sta1=np.mat(expt_n_t_sta1)
for i in range(L//2):
    plt.plot(np.linspace(0.1,20,200),expt_n_t_sta1[:,2*i],label=i)
plt.legend(loc=[1.01,0])
plt.ylim([-0.52,0.52])

import pandas as pd


df = pd.DataFrame(expt_n_t_sta1)

df.to_csv('neel4f.csv', index=False)

In [ ]:
###effective model
L=18# system size
basis = spin_basis_1d(L,pauli=False,m=0) 
#i0 = basis.index("0110010101") # pick state from basis set


i0 = basis.index("011001100110011001") # pick state from basis set
psi01 = np.zeros(basis.Ns,dtype=np.float64)
psi01[i0] = 1.0 # define state corresponding to the string "111000"
psi0=(psi01)

Jr=1# spin ring
##### construct basis in the 0-total momentum and +1-parity sector
# define PBC site-coupling lists for operators
H0=[[-Jr,  4*i,(4*i+1),(4*i+2),(4*i+3)]  for i in range(L//4)] 

H1=[[-Jr,  4*i+2,(4*i+3),(4*i+4)%L,(4*i+5)%L]  for i in range(L//4)] 
print(H0,H1)
no_checks = dict(check_herm=False,check_symm=False,check_pcon=False)
# static and dynamic lists
static0=[["+--+",H0],["-++-",H0],["+--+",H1],["-++-",H1]]

dynamic=[]
###### construct Hamiltonian
Hd0=hamiltonian(static0,dynamic,basis=basis,**no_checks)
#
ttol=10
step=100
tlist1=np.linspace(0,ttol,step)

psi_list=Hd0.evolve(psi0,0,tlist1)

n_list = [hamiltonian([["z",[[1.0,i]]]],[],basis=basis,dtype=np.float64,**no_checks) for i in range(L)]
expt_n_t_sta0 = np.vstack([n.expt_value(psi_list).real for n in n_list]).T
#print(expt_n_t_sta,psi_k)
for i in range(L//2):
    plt.plot(tlist1,expt_n_t_sta0[:,2*i],label=i)
plt.legend(loc=[1.01,0])
plt.ylim([-0.52,0.52])
aa=np.zeros(step)
for i in range(L//2):
    aa+=expt_n_t_sta0[:,2*i]
plt.plot(tlist1,aa)
plt.ylim([-0.52,0.52])
aaa=[]
for i in range(L):
    aaa.append(expt_n_t_sta0[:,i])
import pandas as pd


df = pd.DataFrame(aaa)

df.to_csv('neel4e.csv', index=False)

In [ ]:
##XX model
L=18# system size
basis = spin_basis_1d(L,pauli=False,m=0) 
#i0 = basis.index("0110010101") # pick state from basis set


i0 = basis.index("011001100110011001") # pick state from basis set
psi01 = np.zeros(basis.Ns,dtype=np.float64)
psi01[i0] = 1.0 # define state corresponding to the string "111000"
psi0=(psi01)

Jp=1# spin ring
h=0
##### construct basis in the 0-total momentum and +1-parity sector
# define PBC site-coupling lists for operators
H0=[[-Jp,i,(i+2)%L]  for i in range(L)] 
H1=[[-Jp,2*i,(2*i+1)]  for i in range(L//2)] 


H01=[[h*i,2*i] for i in range(L//2)]
H02=[[h*(i+1),2*i+1] for i in range(L//2)]

print(H01,H02)
no_checks = dict(check_herm=False,check_symm=False,check_pcon=False)
# static and dynamic lists
static0=[["+-",H0],["-+",H0],["+-",H1],["-+",H1], ["z",H01],["z",H02]]

dynamic=[]
###### construct Hamiltonian
Hd0=hamiltonian(static0,dynamic,basis=basis,**no_checks)
#
ttol=10
step=100
tlist1=np.linspace(0,ttol,step)

psi_list=Hd0.evolve(psi0,0,tlist1)

n_list = [hamiltonian([["z",[[1.0,i]]]],[],basis=basis,dtype=np.float64,**no_checks) for i in range(L)]
expt_n_t_sta1 = np.vstack([n.expt_value(psi_list).real for n in n_list]).T
#print(expt_n_t_sta,psi_k)
for i in range(L//2):
    plt.plot(tlist1,expt_n_t_sta1[:,2*i],label=i)
plt.legend(loc=[1.01,0])
plt.ylim([-0.52,0.52])
aa=np.zeros(step)
for i in range(L//2):
    aa+=expt_n_t_sta1[:,2*i]
plt.plot(tlist1,aa)
aaa=[]
for i in range(L):
    aaa.append(expt_n_t_sta1[:,i])
import pandas as pd


df = pd.DataFrame(aaa)

df.to_csv('neel2.csv', index=False)